In [ ]:
options(warn = -1, verbose=FALSE)
#!/usr/bin/env Rscript 
library(dplyr)
library(httr)
library(pheatmap)
library(ggplot2)
library(unixtools)
library(repr)
library(ggmin)
library(purrr)
library(geojsonio)
library(circlize)
library(pryr)
set_config(config(ssl_verifypeer = 0L))
ulimit::memory_limit(100000)
set.tempdir("/datastore/lucy/tmp/")
setwd("/datastore/lucy/CosMx")

## Load Korsunksy lab functions

In [ ]:
source("./R/utils.R")
start_upR(TRUE)
source("./R/TissueSegFunctions.R")

## Load cell geometry and annotations

In [ ]:
objH<-readRDS("./cache/tissueSegmentation/spatialObjregions.RDS")

In [ ]:
head(objH$metadata)

In [ ]:
niche_anno<-objH$metadata %>% dplyr::select(cellID,highres.regions) 
head(niche_anno)

In [ ]:
rm(objH)

In [ ]:
cellgeoms<-readRDS("./cache/raw/Glasgow_cellgeoms_QC.RDS") 

In [ ]:
head(cellgeoms, 2)

In [ ]:
cell_ann<-readRDS("cache/CosMxcoarseGrainharmonyObj_markers.RDS")$metadata

In [ ]:
head(cell_ann, 2)

In [ ]:
cell_ann <- cell_ann %>% left_join(niche_anno, by = "cellID")

In [ ]:
head(cell_ann, 2)

In [ ]:
unique(cell_ann$fine.types)

In [ ]:
sampleFOVcondition <- read.table("/datastore/lucy/CosMx/cache/SampleFOV-condition.csv", header=TRUE, sep=",")

In [ ]:
#Combine with info about condition of each FOV
cellgeoms <- merge(cellgeoms, sampleFOVcondition, all=FALSE)
cell_ann <- merge(cell_ann, sampleFOVcondition, all=FALSE)

In [ ]:
cellgeoms$SampleFOV <- paste0(cellgeoms$Condition,"_",cellgeoms$SampleFOV)
cell_ann$SampleFOV <- paste0(cell_ann$Condition,"_",cell_ann$SampleFOV)

In [ ]:
pixel_size <- (0.18^2) #This is incorrect

In [ ]:
pixel_size

In [ ]:
mean(cellgeoms$area_um) 

In [ ]:
cell_ann$dc.t <- ifelse(cell_ann$fine.types%in%c('Myeloid_DC1 CLEC9A+',
                                 'Myeloid_DC2 CD1c+',
                                 'Myeloid_DC2 CCR7+',
                                 'Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_iDC3 CD14highCD163+',
                                 'Myeloid_FOLR2highCLEC10A+ STM',"Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs'), as.character(cell_ann$fine.types), as.character(cell_ann$new.coarse.types))

In [ ]:
unique(cell_ann$dc.t)

In [ ]:
cell_ann$dc.t <- as.ordered(factor(cell_ann$dc.t, 
                        levels=c("Adipocytes","PRG4+ Lining Fibroblast","Fibroblast",
                                 "Mural","Endothelial","Plasma","B cells",
                                 "Tcell_CD55+PDE4B+CD4+ Naive","Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs',"CD8+ Tcell","NK cell","Mast","Macrophage",
                                 'Myeloid_FOLR2highCLEC10A+ STM','Myeloid_iDC3 CD14highCD163+','Myeloid_DC3 CD1clowCD163+',
                                 'Myeloid_DC2 CD1c+','Myeloid_DC2 CCR7+','Myeloid_DC1 CLEC9A+'
                                 )))

In [ ]:
levels(cell_ann$dc.t)

In [ ]:
t.cell.clusts <- c("Tcell_CD55+PDE4B+CD4+ Naive",
                                 "Tcell_CCR7+CD7+CD4+ Naive/TCM",
                                 'Tcell_BAG3+DNAJB1+CD4+ T',
                                 'Tcell_CCR6+RORA+TNFSF13B+ TEM',
                                  'Tcell_CCL5+GZMA+CD4+ TEM',
                                 'Tcell_CCL5+CXCR6+MAF+ TPH',
                                 'Tcell_CXCL13hi MAF+ TPH',
                                 'Tcell_FOXP3+ Tregs')

In [ ]:
t.cols <- c("Tcell_CD55+PDE4B+CD4+ Naive"='#AFE1AF',
"Tcell_CCR7+CD7+CD4+ Naive/TCM"="#7CFC00",
'Tcell_BAG3+DNAJB1+CD4+ T'="Purple",
                         'Tcell_CCR6+RORA+TNFSF13B+ TEM'="darkorange",
                                                      'Tcell_CCL5+GZMA+CD4+ TEM'='#FF3131',
                                                                         'Tcell_CCL5+CXCR6+MAF+ TPH'="dodgerblue2",
                                                                         'Tcell_CXCL13hi MAF+ TPH'="darkturquoise",
                                                                         'Tcell_FOXP3+ Tregs'="#FFFF00")

In [ ]:
unique(activeRA_cell_ann$highres.regions)

## cDC and CD4 T cell

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann <- cell_ann[which(cell_ann$Condition%in%"Active RA"),]
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "cDC"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$new.coarse.types%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))

In [ ]:
dim(index_neighbours)
head(index_neighbours)

In [ ]:
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)

In [ ]:
index_neighbours.melt <- melt(index_neighbours)

In [ ]:
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
index_neighbours.tcell.melt_activeRA <- index_neighbours.melt[which(index_neighbours.melt$neighbour_coarse%in%"CD4+ Tcell"),]

In [ ]:
index_neighbours.tcell.melt_activeRA$condition <- "Active RA"

In [ ]:
unique(cell_ann$Condition)

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- cell_ann[which(cell_ann$Condition%in%"RemissionRA"),]
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "cDC"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$new.coarse.types%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))

In [ ]:
dim(index_neighbours)
head(index_neighbours)

In [ ]:
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)

In [ ]:
index_neighbours.melt <- melt(index_neighbours)

In [ ]:
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
index_neighbours.tcell.melt_remRA <- index_neighbours.melt[which(index_neighbours.melt$neighbour_coarse%in%"CD4+ Tcell"),]

In [ ]:
index_neighbours.tcell.melt_remRA$condition <- "Remission RA"

In [ ]:
index_neighbours.tcell.melt <- rbind(index_neighbours.tcell.melt_activeRA, index_neighbours.tcell.melt_remRA)

In [ ]:
fig.size(6, 10)
ggplot(index_neighbours.tcell.melt_remRA) + geom_freqpoly(aes(distance, color=condition), bins=10, size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=30),axis.text=element_text(size=20),axis.title=element_text(size=25)) + ggtitle(paste0(index_type)) + xlab("Distance to CD4 T cell neighbour") + ylab("Neighbour count") + scale_color_manual(values=c("blue"))

In [ ]:
fig.size(6, 10)
ggplot(index_neighbours.tcell.melt) + geom_freqpoly(aes(distance, color=condition), bins=10, size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=30),axis.text=element_text(size=20),axis.title=element_text(size=25)) + ggtitle(paste0(index_type)) + xlab("Distance to CD4 T cell neighbour") + ylab("Neighbour count") + scale_color_manual(values=c("red","blue"))

In [ ]:
fig.size(3, 6)
ggplot(index_neighbours.tcell.melt) + geom_freqpoly(aes(distance, color=condition), bins=10, size=2) + scale_y_log10() + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=20),axis.text=element_text(size=20),legend.text=element_text(size=20),legend.title=element_blank(),axis.title=element_text(size=15)) + ggtitle("Myeloid DC") + xlab("Distance to CD4 T cell neighbour (um)") + ylab("Neighbour count (log10)") + scale_color_manual(values=c("red","blue"))

## Myeloid_DC2 CCR7+ in Active RA 

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "Myeloid_DC2 CCR7+"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$fine.type%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))
dim(index_neighbours)
head(index_neighbours)

In [ ]:
#Add cell metadata
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)
#Melt matrix into dataframe
index_neighbours.melt <- melt(index_neighbours)
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
#Remove pairs with distance 0 i.e. self to self or too far
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
#Subset pairs with T cell neighbours
index_neighbours.tcell.melt <- index_neighbours.melt[which(index_neighbours.melt$neighbour_fine.type%in%t.cell.clusts),]

In [ ]:
fig.size(3, 8)
ggplot(index_neighbours.tcell.melt)  + geom_freqpoly(aes(distance, color=neighbour_fine.type), binwidth=10,size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=15),axis.text=element_text(size=20),axis.title=element_text(size=15)) + ggtitle(paste0(index_type)) + xlab("Distance to CD4 T cell neighbour (um)") + ylab("Neighbour count") + scale_color_manual(values=t.cols)

## Myeloid_DC2 CCR7+ in Active RA Lymphoid

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann[activeRA_cell_ann$highres.regions=="Lymphoid",]
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "Myeloid_DC2 CCR7+"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$fine.type%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))
dim(index_neighbours)
head(index_neighbours)

In [ ]:
#Add cell metadata
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)
#Melt matrix into dataframe
index_neighbours.melt <- melt(index_neighbours)
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
#Remove pairs with distance 0 i.e. self to self or too far
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
#Subset pairs with T cell neighbours
index_neighbours.tcell.melt <- index_neighbours.melt[which(index_neighbours.melt$neighbour_fine.type%in%t.cell.clusts),]

In [ ]:
fig.size(3, 8)
ggplot(index_neighbours.tcell.melt)  + geom_freqpoly(aes(distance, color=neighbour_fine.type), binwidth=10,size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=15),axis.text=element_text(size=20),axis.title=element_text(size=15)) + ggtitle(paste0(index_type," in Lymphoid")) + xlab("Distance to CD4 T cell neighbour (um)") + ylab("Neighbour count") + scale_color_manual(values=t.cols)

## Myeloid_iDC3 CD14highCD163+ in Active RA 

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "Myeloid_iDC3 CD14highCD163+"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$fine.type%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))
dim(index_neighbours)
head(index_neighbours)

In [ ]:
#Add cell metadata
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)
#Melt matrix into dataframe
index_neighbours.melt <- melt(index_neighbours)
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
#Remove pairs with distance 0 i.e. self to self or too far
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
#Subset pairs with T cell neighbours
index_neighbours.tcell.melt <- index_neighbours.melt[which(index_neighbours.melt$neighbour_fine.type%in%t.cell.clusts),]

In [ ]:
fig.size(3, 8)
ggplot(index_neighbours.tcell.melt)  + geom_freqpoly(aes(distance, color=neighbour_fine.type), binwidth=10,size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=15),axis.text=element_text(size=20),axis.title=element_text(size=15)) + ggtitle(paste0(index_type)) + xlab("Distance to CD4 T cell neighbour (um)") + ylab("Neighbour count") + scale_color_manual(values=t.cols)

## Myeloid_iDC3 CD14highCD163+ in Active RA Myeloid-rich Lining

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann[activeRA_cell_ann$highres.regions=="Lining_Myeloid-rich",]
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "Myeloid_iDC3 CD14highCD163+"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$fine.type%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))
dim(index_neighbours)
head(index_neighbours)

In [ ]:
#Add cell metadata
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)
#Melt matrix into dataframe
index_neighbours.melt <- melt(index_neighbours)
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
#Remove pairs with distance 0 i.e. self to self or too far
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
#Subset pairs with T cell neighbours
index_neighbours.tcell.melt <- index_neighbours.melt[which(index_neighbours.melt$neighbour_fine.type%in%t.cell.clusts),]

In [ ]:
fig.size(3, 8)
ggplot(index_neighbours.tcell.melt)  + geom_freqpoly(aes(distance, color=neighbour_fine.type), binwidth=10,size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=15),axis.text=element_text(size=20),axis.title=element_text(size=15)) + ggtitle(paste0(index_type," in Myeloid-rich lining")) + xlab("Distance to CD4 T cell neighbour (um)") + ylab("Neighbour count") + scale_color_manual(values=t.cols)

## Myeloid_FOLR2highCLEC10A+ STM in Active RA

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "Myeloid_FOLR2highCLEC10A+ STM"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$fine.type%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))
dim(index_neighbours)
head(index_neighbours)

In [ ]:
#Add cell metadata
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)
#Melt matrix into dataframe
index_neighbours.melt <- melt(index_neighbours)
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
#Remove pairs with distance 0 i.e. self to self or too far
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
#Subset pairs with T cell neighbours
index_neighbours.tcell.melt <- index_neighbours.melt[which(index_neighbours.melt$neighbour_fine.type%in%t.cell.clusts),]

In [ ]:
fig.size(6, 10)
ggplot(index_neighbours.tcell.melt) + geom_freqpoly(aes(distance, color=neighbour_fine.type), size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=15),axis.text=element_text(size=20),axis.title=element_text(size=25)) + ggtitle(paste0(index_type)) + xlab("Distance to CD4 T cell neighbour") + ylab("Neighbour count") + scale_color_manual(values=t.cols)

## Myeloid_FOLR2highCLEC10A+ STM in Active RA Sublining_Myeloid-rich

In [ ]:
#Isolate cells in region of interest and get centroid coords
annotations <- activeRA_cell_ann[activeRA_cell_ann$highres.regions=="Sublining_Myeloid-rich",]
coords <- annotations[, .(x, y)]

In [ ]:
#Get index cell type of interest
index_type <- "Myeloid_FOLR2highCLEC10A+ STM"

In [ ]:
#Calculate nearest neighbours, return_weights=TRUE returns distances
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
mean(adj@x)
quantile(adj@x, c(.05, .95))
head(adj,1)

In [ ]:
#Add cell IDs
rownames(adj) <- annotations$cellID
colnames(adj) <- annotations$cellID
dim(adj)

In [ ]:
#Subset matrix to contain only neighbours
i_index <- which(annotations$fine.type%in%index_type)
index_neighbours = as.data.frame(t(as.matrix(adj[i_index, ])))
dim(index_neighbours)
head(index_neighbours)

In [ ]:
#Add cell metadata
index_neighbours$new.coarse.types <- annotations$new.coarse.types
index_neighbours$fine.type <- annotations$fine.type
index_neighbours$neighbour <- rownames(index_neighbours)
#Melt matrix into dataframe
index_neighbours.melt <- melt(index_neighbours)
index_neighbours.melt$index_fine.type <- index_type

In [ ]:
colnames(index_neighbours.melt) <- c("neighbour_coarse","neighbour_fine.type","neighbour","index","distance","index_type")
head(index_neighbours.melt)

In [ ]:
#Remove pairs with distance 0 i.e. self to self or too far
index_neighbours.melt <- index_neighbours.melt[index_neighbours.melt$distance>0,]

In [ ]:
#Subset pairs with T cell neighbours
index_neighbours.tcell.melt <- index_neighbours.melt[which(index_neighbours.melt$neighbour_fine.type%in%t.cell.clusts),]

In [ ]:
fig.size(6, 10)
ggplot(index_neighbours.tcell.melt) + geom_freqpoly(aes(distance, color=neighbour_fine.type), bins=10, size=2) + 
    guides(fill=FALSE) + xlim(0,100) + theme_powerpoint() + 
    theme(plot.title=element_text(size=15),axis.text=element_text(size=20),axis.title=element_text(size=25)) + ggtitle(paste0(index_type," in Sublining Myeloid-rich")) + xlab("Distance to CD4 T cell neighbour") + ylab("Neighbour count") + scale_color_manual(values=t.cols)

## Distance of all cells regardless of neighbour

In [ ]:
activeRA_cell_ann <- activeRA_cell_ann[which(!is.na(activeRA_cell_ann$dc.t)),]

In [ ]:
unique(activeRA_cell_ann$dc.t)

In [ ]:
unique(activeRA_cell_ann$new.coarse.types)

In [ ]:
coords <- activeRA_cell_ann[, .(x, y)]

In [ ]:
adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
    adj@x[adj@x > 40] = 0
    adj = Matrix::drop0(adj)
    adj@x = rep(1, length(adj@x))

In [ ]:
cellgeoms <- readRDS("./cache/cellgeoms_finetyped.RDS")

In [ ]:
fov.list <- as.list(unique(cellgeoms$SampleFOV))

In [ ]:
head(cellgeoms)

In [ ]:
unique(cellgeoms$fine.types)

In [ ]:
#Determine cell centroid i.e. centre point of each polygon
cellgeoms <- cellgeoms %>%
    dplyr::mutate(centroid = st_centroid(geometry)) 

In [ ]:
dc_cents <- list()
t_cents <- list()
distance <- list()
distance.data <- list()

In [ ]:
findDistance <- function(pairs){
    index = pairs$Var1
    neighbor = pairs$Var2
    index_coords = st_coordinates(index)
    neighbor_coords = st_coordinates(neighbor)
    distance = sqrt((index_coords[, 1] - neighbor_coords[, 1])^2 + (index_coords[, 2] - neighbor_coords[, 2])^2)
    return(distance)
}

In [ ]:
activeRA_geoms <- cellgeoms[which(cellgeoms$Condition=="Active RA"),]

In [ ]:
fov.list <- unique(activeRA_geoms$SampleFOV)
dc.cluster <- "Myeloid_iDC3 CD14highCD163+"
t.cluster <- "Tcell_CCL5+CXCR6+MAF+ TPH"

for(i in 1:length(fov.list)) {
    
    #Subset cell geometries to FOV of interest
    fov <- fov.list[[i]]
    print(fov)
    fov_cellgeoms <- activeRA_geoms[which(activeRA_geoms$SampleFOV==fov),]
    
    #If DC cluster is present in FOV
    if(length(which(fov_cellgeoms$fine.types==dc.cluster))>0){
     if(length(which(fov_cellgeoms$fine.types==t.cluster))>0){
        print("Calculating distance")
        
        #Subset cell geometries for DC and T cell clusters of interest
        dc <- as.data.frame(fov_cellgeoms[which(fov_cellgeoms$fine.types==dc.cluster),c("centroid")])
        t <- as.data.frame(fov_cellgeoms[which(fov_cellgeoms$fine.types==t.cluster),c("centroid")])
         
        #Determine all possible combinations of DC and T cell interaction 
        pairs <- expand.grid(dc$geometry,t$geometry)
      
        #Calculate distance between centroids 
        dists <- findDistance(pairs)
                    
        print("done")    
        
    } else { 
        print ("T cluster not present") } }
    else { 
        print ("DC cluster not present") }

        distance.data[[fov]] <- dists

}

In [ ]:
plot <- as.data.frame(unlist(distance.data))

In [ ]:
colnames(plot) <- "distance"

In [ ]:
head(plot)

In [ ]:
fig.size(5, 8)
ggplot(plot) + geom_histogram(aes(distance),binwidth = 10) 

In [ ]:
fov.list <- unique(activeRA_geoms$SampleFOV)
dc.cluster <- "Myeloid_iDC3 CD14highCD163+"
t.cluster <- "Tcell_CXCL13hi MAF+ TPH'"

for(i in 1:length(fov.list)) {
    
    #Subset cell geometries to FOV of interest
    fov <- fov.list[[i]]
    print(fov)
    fov_cellgeoms <- activeRA_geoms[which(activeRA_geoms$SampleFOV==fov),]
    
    #If DC cluster is present in FOV
    if(length(which(fov_cellgeoms$fine.types==dc.cluster))>0){
     if(length(which(fov_cellgeoms$fine.types==t.cluster))>0){
        print("Calculating distance")
        
        #Subset cell geometries for DC and T cell clusters of interest
        dc <- as.data.frame(fov_cellgeoms[which(fov_cellgeoms$fine.types==dc.cluster),c("centroid")])
        t <- as.data.frame(fov_cellgeoms[which(fov_cellgeoms$fine.types==t.cluster),c("centroid")])
         
        #Determine all possible combinations of DC and T cell interaction 
        pairs <- expand.grid(dc$geometry,t$geometry)
      
        #Calculate distance between centroids 
        dists <- findDistance(pairs)
                    
        print("done")    
        
    } else { 
        print ("T cluster not present") } }
    else { 
        print ("DC cluster not present") }

        distance.data[[fov]] <- dists

}

In [ ]:
plot <- as.data.frame(unlist(distance.data))

In [ ]:
colnames(plot) <- "distance"

In [ ]:
head(plot)

In [ ]:
fig.size(5, 8)
ggplot(plot) + geom_histogram(aes(distance),binwidth = 10) #+ scale_x_log10() 

In [ ]:
 #Subset cell geometries for DC and T cell clusters of interest
fov_cellgeoms <- as.data.frame(fov_cellgeoms[which(fov_cellgeoms$fine.types%in%c(dc.cluster,t.cluster)),
                                             c("centroid","fine.types")])

In [ ]:
head(fov_cellgeoms)

In [ ]:
lines <- st_sfc(mapply(function(a,b){st_cast(st_union(a,b),"LINESTRING")}, dc$geometry, t$geometry, SIMPLIFY=FALSE))

In [ ]:
ggplot(fov_cellgeoms) + geom_sf(aes(geometry=centroid, color=fine.types)) + scale_color_manual(values=c("red","blue")) 